In [1]:
import numpy as np
import pandas as pd


In [2]:
data=pd.read_csv('train_E6oV3lV.csv')
test_data=pd.read_csv('test_tweets_anuFYb8.csv')

In [3]:
data.head(10)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...


In [4]:
data.shape

(31962, 3)

In [5]:
# Clean the data by removing @ , # etc
import re
def clean_text(df, text_field):
    df[text_field]=df[text_field].str.lower()
    df[text_field]=df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(w+:\/\/\S+)|^rt|http.+?", "", elem))
    return df

train=clean_text(data,"tweet")
test=clean_text(test_data,"tweet")

In [6]:
train.head(10)

,id,label,tweet
0,1,0,when a father is dysfunctional and is so sel...
1,2,0,thanks for lyft credit i cant use cause they...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in u...
4,5,0,factsguide society now motivation
5,6,0,22 huge fan fare and big talking before they l...
6,7,0,camping tomorrow danny
7,8,0,the next school year is the year for exams can...
8,9,0,we won love the land allin cavs champions clev...
9,10,0,welcome here im its so gr8


In [7]:
# The data with label 1 is very small. So, we have to use upsampling and downsampling.
# Upsampling- The same data with label 1 will be used again and again.
# The data with majority class will be used without replacement

#Upsampling
from sklearn.utils import resample

train_majority=train[train.label==0]
train_minority=train[train.label==1]

train_minority_upsampled=resample(train_minority, replace=True, n_samples=len(train_majority), random_state=123)
train_upsampled=pd.concat([train_minority_upsampled,train_majority])
train_upsampled['label'].value_counts()


1    29720
0    29720
Name: label, dtype: int64

In [8]:
# Downsampling
from sklearn.utils import resample

train_majority=train[train.label==0]
train_minority=train[train.label==1]

train_majority_downsampled=resample(train_majority, replace=True, n_samples=len(train_minority), random_state=123)
train_downsampled=pd.concat([train_minority, train_majority_downsampled])
train_downsampled['label'].value_counts()

1    2242
0    2242
Name: label, dtype: int64

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

pipeline_sgd=Pipeline([('vect', CountVectorizer()),('tfidf',TfidfTransformer()),('nb',SGDClassifier()),])

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(train_upsampled['tweet'], train_upsampled['label'], random_state=0)

In [11]:
X_train.shape, y_train.shape, X_test.shape

((44580,), (44580,), (14860,))

In [12]:
model=pipeline_sgd.fit(X_train, y_train)
y_predict=model.predict(X_test)

from sklearn.metrics import f1_score
f1_score(y_test,y_predict)

/home/raman/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.9693333333333334

In [13]:
test.head(10)

,id,tweet
0,31963,studiolife aislife requires passion dedication...
1,31964,white supremacists want everyone to see the ...
2,31965,safe ways to heal your acne altwaystoheal h...
3,31966,is the hp and the cursed child book up for res...
4,31967,3rd bihday to my amazing hilarious nephew el...
5,31968,choose to be momtips
6,31969,something inside me dies eyes ness smokeyeye...
7,31970,finishedtattooinkedinkloveit thanksaleeee
8,31971,i will never understand why my dad left me...
9,31972,delicious food lovelife capetown mannaepicur...


In [14]:
test['label']=model.predict(test.tweet)

In [15]:
test.head(30)

,id,tweet,label
0,31963,studiolife aislife requires passion dedication...,0
1,31964,white supremacists want everyone to see the ...,1
2,31965,safe ways to heal your acne altwaystoheal h...,0
3,31966,is the hp and the cursed child book up for res...,0
4,31967,3rd bihday to my amazing hilarious nephew el...,0
5,31968,choose to be momtips,0
6,31969,something inside me dies eyes ness smokeyeye...,0
7,31970,finishedtattooinkedinkloveit thanksaleeee,0
8,31971,i will never understand why my dad left me...,0
9,31972,delicious food lovelife capetown mannaepicur...,0


In [16]:
data_submit=test[['id','label']]
data_submit.set_index('id', inplace=True)
data_submit.to_csv('data_submit.csv')

In [26]:
X_train1, y_train1= train_upsampled['tweet'], train_upsampled['label']
model1=pipeline_sgd.fit(X_train1, y_train1)
test['label']=model1.predict(test.tweet)

In [27]:
test.head(30)

,id,tweet,label,label1
0,31963,studiolife aislife requires passion dedication...,0,0
1,31964,white supremacists want everyone to see the ...,1,1
2,31965,safe ways to heal your acne altwaystoheal h...,0,0
3,31966,is the hp and the cursed child book up for res...,0,0
4,31967,3rd bihday to my amazing hilarious nephew el...,0,0
5,31968,choose to be momtips,0,0
6,31969,something inside me dies eyes ness smokeyeye...,0,0
7,31970,finishedtattooinkedinkloveit thanksaleeee,0,0
8,31971,i will never understand why my dad left me...,0,0
9,31972,delicious food lovelife capetown mannaepicur...,0,0


In [28]:
data_submit=test[['id','label']]
data_submit.set_index('id', inplace=True)
data_submit.to_csv('data_submit.csv')